## 9.1 Stock Data
1. Scope:
    - have data on stock prices (open, close, high, low)
    - have 1000 client apps accessing this data
2. Assumptions:
    - data is going to be read-only. don't want client apps to change the prices
    - they might be requesting the data only every couple of hours since i assume stock prices don't change that much and they are asking for end-of-day stock price info
3. Major Components:
    - database to contain all of this info
        - probably going to use a SQL-based DBMS like MySQL for the job. since it's mostly going to be read-only queries involved, don't need the DBMS to have complicated queries or actions
    - going to have some sort of API to handle all client requests and to query database
        - going to use node.js to create it since you can make clusters of the instance if traffic is heavy but it is simple enough for what we're doing
4. Key Issues:
    - if we just run one instance of the node.js api then it can be a big bottleneck if all of the clients make the requests within a small time period
    - if we only have one database with all the info, then it could also be a source of a bottleneck if there are multiple queries going on
5. Redesign:
    - we can scale the api by using node.js clusters which will have clusters that can divide the traffic and handle all of the requests simultaneously. decent solution if we make use of a multi-core system that can do this
    - we could have clones of the database that the clusters could access. it will also divide up the queries

### Book Solutions:

## 9.2 Social Network

1. Scope:
    - what features should this social network have? 
        - profiles, connections, messages, etc
    - who is the target audience?
        - linkedin and facebook cater to different user bases.
        - one is for professionals looking for jobs or posting up job offers
        - the other is a way for families, friends, acquaintances to connect
        - identifying the user audience is important to determine what features it should have
2. Assumptions:
    - going to assume a generic facebook-like social network
    - going to have a user profile, friends list, etc
    - able to message between user and their friends either 1-1 or group chat
    - going to have some sort of a timeline for user's to create posts and share them
3. Major components:
    - user profile
    - friends list
    - messaging system
    - timeline
    - data structure to use: a graph
        - a network itself is like a big graph so it's a no-brainer for the social network to be one as well
        - the social network itself is going to be a large graph with each of its nodes acting as a feature
            - so one node for profiles
            - one node for messaging system
            - one node for timelines etc
        - the user profiles themselves are graphs with each node being assign a different function
            - one node for mutuals
            - one node for user's information
            - another node for posts
            - and one for any messages
4. Key issues:
    - might have redundant information in each graph and will be pretty tedious to update them
    - so we have to update the messages in the user profile graph and in the messaging system node in the social network graph
5. Redesign:
    - should just keep that information in one place to make it more efficient to update
    - so all messages should be kept in the messaging system node and the user's messaging node has a reference to all their conversations
***
* for the shortest path between two users based on their mutuals:
    1. should be using a breadth-first search to find the shortest path
    2. essentially you run a double bfs until you land on a mutual user between the two
        - so you run bfs from user1 --> user2
        - and you simultaneously rn one from user2 --> user1
        - if the algorithm lands on the same mutual or on one that the other already looked through, then you have found the shortest path
    3. the way the data structure is set up for the social network as a big graph will help tremendously in this b/c you will have a list of friends/mutuals to work with for the double-bfs method
* a possible optimization would be to first look through all the connections of user1 and check if they are also user 2's connections
    - if there is 1 match, then that mutual is the shortest path between the two users and the double-bfs method is not necessary
    - it would probably be faster to do this than jumping straight into double-bfs b/c if both of the users have huge amounts of connections, like 5k+, then that is a lot of mutuals to go through

### Book Solutions:

## 9.3 Web Crawler

1. Scope:
    - what are the infinite loops in regards to?
    - is there a possibility for infinite loops when crawling within a domain's contents or are we talking about infinite loops when crawling through the entire web?
        - the answer to this could change the way the algorithm is designed
2. Assumptions:
    - assume infinite looping in regards to pages within a site so mypage/about vs mypage/faq, etc
    - assume site is a multi-page application rather than a single-page one
3. Major Components:
    - could have a hash table that takes in the URL of the page currently being crawled
    - if the crawler enters a site that it's already seen before, it will abort the operation and move onto other pages or sites
    - within that hash table, each site will also have info on the contents of the site
    - should have another hash table with some info on common html elements like head or body tags to make a comparison in case the URL check fails
4. Key issues:
    - for URLs, there could be sites that have similar URLs but pretty much the same content and that could be accidentally crawled infinitely
    - it also doesn't take into account the same domains/subdomains for a page
5. Redesign:
    - the hash table should also add in info on domains/subdomains that have similar extensions, etc
    - especially for sites that use QUERIES at the end of the URLs which would have different URLs but pretty much the same content

### Book Solutions:

## 9.4 Duplicate URLs

1. Scope:
    - what should we do with the duplicate URLs? should we just notify someone that there is a duplicate URL or should it be removed?
    - how are these URLs gathered and stored in the first place? is there a list of these URLs or do we have to crawl for them?
    - is there a distinction between domains and subdomains?
        - do maps.google.com and google.com count as duplicates?
2. Assumptions:
    - assume that the whole URL has to be unique
        - so google.com/results, google.com, and maps.google.com are three unique results
    - also assume that the URLs are stored in a list of some sort, like an array
3. Algorithm:
    1. want to have iterate through the entire list of URLs and add them to a hash table
    2. while iterating, check to see if the URL is already in the hash table. if it is, then we can notify that it is a duplicate and move on. 
        - this will essentially take O(n) space where n = # of URLs. in this case, n = 10 billion URLs
        - it should only take 1 pass through this list of URLs to notify and find any duplicates b/c as you pass through the list, you only keep track of unique URLs in the hash table and any duplicates will immediately be found
        - it should only take 1 machine to do
4. Key issues:
    - if we wanted to scale this further to more than 10 billion URLs, we might not have enough space to fit all the URLs onto one machine
5. Redesign:
    - we could split up the list of URLs into multiple parts and do the same algorithm on multiple machines
    - then once we are done, we can compare all the URLs in each hash table to see if there are duplicates between the hash tables

### Book Solutions: